# HA-ES Plotting

- udpated version of plotting script used for paper "..."

## General

- imports
- defintions
- loading data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmo as pg
import seaborn as sns

import math
import os

from autorank._util import get_sorted_rank_groups
from autorank import autorank, plot_stats
from tabrepo import load_repository

method_id_name_dict = {
    "GES": "GES*",
    "SINGLE_BEST": "Single-Best",
    "QO": "QO-ES",
    "QDO": "QDO-ES",
    "ENS_SIZE_QDO": "Ensemble Size",
    #"INFER_TIME_QDO": "Inference Time",
    "INFER_TIME_QDO": "HAPEns",
    "MEMORY_QDO": "Memory",
    "DISK_QDO": "Diskspace",
}
infer_time_weights = np.linspace(0, 1, num=20)[1:]
infer_time_weights = np.round(infer_time_weights, 2)
multi_ges_method_ids = [f"MULTI_GES-{time_weight:.2f}" for time_weight in infer_time_weights]
multi_ges_method_names = [f"Multi-GES({time_weight:.2f})" for time_weight in infer_time_weights]
for id, name in zip(multi_ges_method_ids, multi_ges_method_names):
    method_id_name_dict[id] = name

print("Loading data. This might take a while...")
df = pd.read_csv("../data/full.csv")

# Map method IDs to names
method_id_name_dict["MULTI_GES-0.79"] = "Multi-GES"
if "method" in df.columns:
    df["method_name"] = df["method"].map(method_id_name_dict)
else:
    raise ValueError("Column 'method' not found in DataFrame")
df = df.dropna(subset=['method_name'])
print(df.shape)
print(df.columns)
print(df["method_name"].unique())
print(df["method"].unique())

df["models_used_length"] = df["models_used"].apply(len)

Some initial checks...

In [ ]:
single_task_df = df[df['task'] == df['task'].unique()[0]]
print(single_task_df[['normalized_roc_auc_test', 'normalized_time', 'normalized_memory', 'normalized_diskspace']].describe())
df.info()

print("--- Sanity Check: Number of entries per method ---")
entry_counts = df.groupby('method_name').size()
print("Number of entries (solutions) per method:")
print(entry_counts)
print("\\n" + "="*50 + "\\n")


print("--- Sanity Check: Number of unique datasets per method ---")
tasks_per_method = df.groupby('method_name')['task_id'].nunique()
print("Number of unique tasks evaluated per method:")
print(tasks_per_method)
print("\\n" + "="*50 + "\\n")


print("--- Sanity Check: Number of unique seeds per method ---")
seeds_per_method = df.groupby('method_name')['seed'].nunique()
print("Number of unique seeds evaluated per method:")
print(seeds_per_method)
print("\\n" + "="*50 + "\\n")


print("--- Sanity Check: Number of unique folds per method ---")
folds_per_method = df.groupby('method_name')['fold'].nunique()
print("Number of unique folds evaluated per method:")
print(folds_per_method)
print("\\n" + "="*50 + "\\n")

# --- Verification ---
# Check if all methods have the same number of unique tasks, seeds, and folds
all_tasks_consistent = tasks_per_method.nunique() == 1
all_seeds_consistent = seeds_per_method.nunique() == 1
all_folds_consistent = folds_per_method.nunique() == 1

if all_tasks_consistent and all_seeds_consistent and all_folds_consistent:
    print("Verification PASSED: All methods were run on the same number of tasks, seeds, and folds.")
else:
    print("Verification FAILED: There is an inconsistency in the number of tasks, seeds, or folds across methods.")

## Helper Functions

### Boxplot:

In [ ]:
import re
import ipywidgets as widgets

def extract_numeric_part(method_name):
    """
    Extracts the numeric part from a method name string. If no numeric part is found, returns None.
    """
    if isinstance(method_name, str):
        match = re.search(r"\((\d*\.?\d+)\)", method_name)
        if match:
            return float(match.group(1))
    return None

def boxplot(
    df: pd.DataFrame,
    y_str: str,
    log_y_scale: bool = False,
    log_x_scale: bool = False,
    flip_y_axis: bool = False,
    orient: str = "v",
    rotation_x_ticks: int = 45,
    outliers=False,
    sort_by_median: bool = True,  # <-- added flag
):
    if y_str not in df.columns:
        raise ValueError(f"Column '{y_str}' not found in DataFrame")
    
    df["method_name"] = df["method_name"].astype(str)

    if sort_by_median:
        # Sort by median of the target column
        medians = (
            df.groupby("method_name")[y_str]
            .median()
            .sort_values(ascending=False)  # largest median first
        )
        df["method_name"] = pd.Categorical(df["method_name"], categories=medians.index, ordered=True)
        df = df.sort_values("method_name")
    else:
        # Alphabetic + numeric sorting as fallback
        df["alphabetic"] = df["method_name"].apply(lambda x: re.split(r"\(\d*\.?\d+\)", x)[0])
        df["numeric"] = df["method_name"].apply(extract_numeric_part)
        df = df.sort_values(by=["alphabetic", "numeric"], ascending=[True, True])
        df = df.drop(columns=["alphabetic", "numeric"])

    plt.figure(figsize=(8, 6))
    if orient == "v":
        sns.boxplot(
            data=df,
            x="method_name",
            y=y_str,
            hue="method_name",
            palette="pastel",
            linewidth=2,
            orient=orient,
            legend=False,
            showfliers=outliers,
        )
    elif orient == "h":
        sns.boxplot(
            data=df,
            x=y_str,
            y="method_name",
            hue="method_name",
            palette="pastel",
            linewidth=2,
            orient=orient,
            legend=False,
            showfliers=outliers,
        )
    else:
        raise ValueError(f"Orient '{orient}' not supported")

    if orient == "v":
        plt.ylabel(y_str)
        plt.xlabel("Ensemble Method")
    else:
        plt.xlabel(y_str)
        plt.ylabel("Ensemble Method")

    if log_y_scale:
        plt.yscale("log")
    if log_x_scale:
        plt.xscale("log")
    if flip_y_axis:
        plt.gca().invert_yaxis()

    plt.xticks(rotation=rotation_x_ticks)
    plt.grid(True)
    plt.tight_layout()

    directory = "../plots"
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig(f"{directory}/boxplot_{y_str}.png", dpi=300, bbox_inches="tight")
    plt.savefig(f"{directory}/boxplot_{y_str}.pdf", dpi=300, bbox_inches="tight")
    plt.show()

def interactive_boxplot(df, column_options):
    y_str_widget = widgets.Dropdown(
        options=column_options,
        description="Y-axis column:",
        style={"description_width": "initial"},
    )
    log_y_scale_widget = widgets.Checkbox(value=False, description="Log-scale Y-axis")
    log_x_scale_widget = widgets.Checkbox(value=False, description="Log-scale X-axis")
    flip_y_axis_widget = widgets.Checkbox(value=False, description="Flip Y-axis")
    orient_widget = widgets.RadioButtons(
        options=["v", "h"],
        value="h",
        description="Orientation:",
        style={"description_width": "initial"},
    )
    rotation_x_ticks_widget = widgets.IntSlider(
        value=0, min=0, max=90, step=5, description="Rotation X-ticks"
    )

    ui = widgets.VBox(
        [
            y_str_widget,
            log_y_scale_widget,
            log_x_scale_widget,
            flip_y_axis_widget,
            orient_widget,
            rotation_x_ticks_widget,
        ]
    )

    out = widgets.interactive_output(
        boxplot,
        {
            "df": widgets.fixed(df),
            "y_str": y_str_widget,
            "log_y_scale": log_y_scale_widget,
            "log_x_scale": log_x_scale_widget,
            "flip_y_axis": flip_y_axis_widget,
            "orient": orient_widget,
            "rotation_x_ticks": rotation_x_ticks_widget,
        },
    )

    display(ui, out)

### Critical Difference:

In [ ]:
def cd_evaluation(
    scores,
    maximize_metric=True,
    plt_title="Critical Difference Plot",
    filename="CriticalDifferencePlot.png",
):
    """
    scores: DataFrame with method names as columns and tasks as rows, each cell contains a score value.
    maximize_metric: Boolean, True if higher values are better.
    output_path: Where to save the plot, if None, plot will not be saved.
    plt_title: Title of the plot.
    """
    # Prepare data
    rank_data = -scores if maximize_metric else scores

    # Run autorank
    result = autorank(rank_data, alpha=0.05, verbose=False, order="ascending")

    # Plot with updated font size
    plt.close("all")
    width = 6
    fig, ax = plt.subplots(figsize=(12, width))
    plt.rcParams.update({"font.size": 20})

    plot_stats(result, ax=ax)
    ax.tick_params(axis="both", labelsize=20)  # Set font size for axis ticks
    labels = [item.get_text() for item in ax.get_xticklabels()]
    ax.set_xticklabels(labels, fontsize=20)  # Adjust fontsize as needed
    plt.tight_layout()

    plt.savefig(filename, bbox_inches="tight", dpi=300)
    plt.close()

    return result


## Data Exploration

In [ ]:
column_options = ["models_used_length", "normalized_roc_auc_test", "normalized_roc_auc_val", "inference_time", "memory", "diskspace", "normalized_memory"]

#filtered_df = df[~df['method_name'].isin(['Multi-GES(0.43)', 'Multi-GES(0.50)'])].copy()
interactive_boxplot(df, column_options)

In [ ]:
import matplotlib.ticker as mticker
# Group the DataFrame by 'method_name' and compute total and unique counts
result_df = df.groupby('method_name').agg(
    total_solutions=('models_used', 'count'),
    unique_solutions=('models_used', 'nunique')
).reset_index()

# Calculate the percentage of unique solutions
result_df['percentage_unique_solutions'] = (
    result_df['unique_solutions'] / result_df['total_solutions'] * 100
)

# Sort the DataFrame by the percentage of unique solutions for clearer visualization
result_df = result_df.sort_values('percentage_unique_solutions', ascending=False).reset_index(drop=True)


color_total = 'steelblue'
color_unique = 'indianred'
fig, ax = plt.subplots(figsize=(12, 8))
ax.bar(result_df['method_name'], result_df['total_solutions'], color=color_total, label='Total Solutions')

# --- Y-axis 1: Total Solutions ---
ax.set_ylabel("Total Solutions", color='black', fontsize=20, labelpad=15)
ax.tick_params(axis='y', labelcolor='black', pad=5, labelsize=18)
ax.grid(axis='y', linestyle='--', color=color_total, alpha=0.4)
ax.yaxis.set_major_locator(mticker.MaxNLocator(nbins=6, integer=True))
ax2 = ax.twinx()
ax2.plot(result_df['method_name'], result_df['percentage_unique_solutions'], color=color_unique, marker='o', linestyle='-', linewidth=3, markersize=10, label='Percentage Unique Solutions')

# --- Y-axis 2: Percentage Unique ---
ax2.set_ylabel('Percentage of Unique Solutions (%)', color='black', fontsize=20, labelpad=15)
ax2.tick_params(axis='y', labelcolor='black', pad=5, labelsize=18)
ax2.grid(False)
ax2.yaxis.set_major_locator(mticker.MaxNLocator(nbins=6))
ax2.yaxis.set_major_formatter(mticker.PercentFormatter())

# --- Common Settings ---
ax.tick_params(axis='x', rotation=45, labelsize=18)

# --- Legend ---
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(
    lines + lines2,
    labels + labels2,
    loc='upper right',
    fontsize=16,
    frameon=True,
    facecolor='white',
    edgecolor='grey'
)

fig.tight_layout()
plt.savefig("../plots/total-vs-unique-ensembles.png", dpi=300)
plt.savefig("../plots/total-vs-unique-ensembles.pdf", dpi=300)
plt.show()

## Pareto Front Eval

### True Pareto Front

In [ ]:
def getParetoFront(df: pd.DataFrame, objectives: list[str], return_mask: bool = False):
    """
    Finds the Pareto-efficient points from a DataFrame.
    
    This function assumes ALL objectives are to be MINIMIZED.

    Args:
        df (pd.DataFrame): DataFrame containing the points.
        objectives (List[str]): List of column names for the objectives.
        return_mask (bool): If True, returns a boolean mask. If False, returns the filtered DataFrame.

    Returns:
        pd.DataFrame or np.ndarray: The Pareto-efficient points or a boolean mask.
    """
    costs = df[objectives].values
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            # Find all points not dominated by c
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            # And keep c itself
            is_efficient[i] = True
    
    return is_efficient if return_mask else df[is_efficient]

    
def calculate_pareto_fronts(df, objectives):
    """
    Calculate the Pareto front points for all solutions in the DataFrame per task_id, seed, and fold.

    Parameters:
    - df: pandas DataFrame containing the data.
    - objectives: list of two objective column names.

    Returns:
    - A nested dictionary with structure {task_id: {seed: {fold: pareto_front_df}}}
    """
    if len(objectives) != 2:
        raise ValueError("Exactly two objectives must be provided.")

    # Initialize a dictionary to store the Pareto fronts
    pareto_fronts = {}

    # Iterate over unique task_ids
    for task_id in df["task_id"].unique():
        pareto_fronts[task_id] = {}

        # Iterate over unique seeds
        for seed in df["seed"].unique():
            pareto_fronts[task_id][seed] = {}

            # Iterate over unique folds
            for fold in df["fold"].unique():
                # Filter the DataFrame for the current task_id, seed, and fold
                df_fold = df[
                    (df["task_id"] == task_id)
                    & (df["seed"] == seed)
                    & (df["fold"] == fold)
                ]

                if df_fold.empty:
                    continue  # Skip if no data for this combination

                # Compute Pareto front for df_fold
                is_efficient = getParetoFront(df_fold, objectives, return_mask=True)
                pareto_front_df = df_fold[is_efficient]

                # Store the Pareto front DataFrame
                pareto_fronts[task_id][seed][fold] = pareto_front_df

    return pareto_fronts

In [ ]:
from process_data import normalize_data

hw_score = 'hw_score'
perf_metric = 'normalized_roc_auc_test'
hw_metrics = ['normalized_time', 'normalized_memory', 'normalized_diskspace']

df['hw_score'] = df[hw_metrics].mean(axis=1)
for task in df["task"].unique():
    mask = df["task"] == task
    if "hw_score" in df.columns:
        df.loc[mask, "normalized_hw_score"] = normalize_data(
            df.loc[mask, "hw_score"]
        )

metric_pairs = [(perf_metric, hw_metric) for hw_metric in hw_metrics]
metric_pairs.append(('normalized_roc_auc_test', 'hw_score'))
metric_pairs.append(('normalized_roc_auc_test', 'normalized_hw_score'))
print(metric_pairs)

### Hypervolume

In [ ]:
def calculate_average_hypervolumes(
    df: pd.DataFrame,
    method_name: str,
    perf_metric: str,
    hw_metrics: list[str],
) -> dict:
    print(f"Calculating for HV for {method_name} and metrics {perf_metric} {hw_metrics}")
    all_metrics = [perf_metric] + hw_metrics
    ref_point = [1.01] + [1.01] * len(hw_metrics)

    df_method = df[df["method_name"] == method_name]
    hypervolumes_per_task = {}

    # Iterate over unique task_ids
    for task_id in df_method["task_id"].unique():
        seed_hypervolumes = []  # Store hypervolumes for each seed

        for seed in df_method["seed"].unique():
            fold_hypervolumes = []  # Store hypervolumes for each fold

            for fold in df_method["fold"].unique():
                df_fold = df_method[
                    (df_method["task_id"] == task_id)
                    & (df_method["seed"] == seed)
                    & (df_method["fold"] == fold)
                ]

                if df_fold.empty:
                    continue

                points = df_fold[all_metrics].copy()
                points_to_maximize = points.values
                hv = pg.hypervolume(points_to_maximize)
                hypervolume = hv.compute(ref_point)
                fold_hypervolumes.append(hypervolume)

            # Average hypervolumes across all folds for the current seed
            if fold_hypervolumes:
                average_fold_hypervolume = np.mean(fold_hypervolumes)
                seed_hypervolumes.append(average_fold_hypervolume)

        # Average the seed-level averages for the current task
        if seed_hypervolumes:
            average_seed_hypervolume = np.mean(seed_hypervolumes)
            hypervolumes_per_task[task_id] = average_seed_hypervolume

    return hypervolumes_per_task


def plot_hypervolumes(all_hypervolumes, title: str, directory: str = "../plots/"):
    # Prepare the data for plotting
    methods = list(all_hypervolumes.keys())  # Method names
    hv_values = [list(all_hypervolumes[method].values()) for method in methods]
    data = []

    # Creating a DataFrame suitable for Seaborn
    for method_index, values in enumerate(hv_values):
        for value in values:
            data.append({"Method": methods[method_index], "Hypervolume": value})
    df = pd.DataFrame(data)

    # Calculate medians for each method and sort by median
    median_order = df.groupby("Method")["Hypervolume"].median().sort_values(ascending=False).index

    # Set the figure size and style
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 6))

    # Use seaborn's boxplot to plot the DataFrame, ordered by median values
    ax = sns.boxplot(
        y="Method", x="Hypervolume", data=df, hue="Method", palette="Set2",
        orient="h", order=median_order
    )

    # Set titles and labels
    ax.set_ylabel("Method", fontsize=20)
    ax.set_xlabel("Hypervolume", fontsize=20)

    # Set font size for ticks
    ax.tick_params(axis="x", labelrotation=45, labelsize=16)
    ax.tick_params(axis="y", labelsize=16)

    plt.tight_layout()
    plt.savefig(directory + title + ".png", dpi=300)
    plt.savefig(directory + title + ".pdf", dpi=300)
    plt.close()

In [ ]:
for perf_metric, hw_metric in metric_pairs:
    methods = df["method_name"].unique()
    all_hypervolumes = {}

    for method in methods:
        all_hypervolumes[method] = calculate_average_hypervolumes(df, method, perf_metric=perf_metric, hw_metrics=[hw_metric])

    # You might want to make this plot function also save to a dynamic filename
    plot_hypervolumes(all_hypervolumes, f"bp_hv_{perf_metric}_{hw_metric}")

    # Create dynamic filenames to avoid overwriting results
    csv_filename = f"../data/hypervolumes_{perf_metric}_{hw_metric}.csv"
    plot_filename_pdf = f"../plots/CDP_HV_{perf_metric}_{hw_metric}.pdf"
    plot_filename_png = f"../plots/CDP_HV_{perf_metric}_{hw_metric}.png"

    hypervolumes_df = pd.DataFrame(all_hypervolumes)
    hypervolumes_df.to_csv(csv_filename, index=False)
    print(f"Saved hypervolume data to {csv_filename}")

    data = []
    for method, task_ids in all_hypervolumes.items():
        for task_id, hypervolume in task_ids.items():
            data.append({"Task": task_id, "Method": method, "Hypervolume": hypervolume})

    df_hypervolumes = pd.DataFrame(data)
    pivot_hypervolumes = df_hypervolumes.pivot(
        index="Task", columns="Method", values="Hypervolume"
    )

    # Run CD evaluation with dynamic filenames
    hv_result = cd_evaluation(
        pivot_hypervolumes,
        maximize_metric=True,
        plt_title=f"Hypervolume CD Plot for {perf_metric} vs {hw_metric}",
        filename=plot_filename_pdf,
    )
    hv_result = cd_evaluation(
        pivot_hypervolumes,
        maximize_metric=True,
        plt_title=f"Hypervolume CD Plot for {perf_metric} vs {hw_metric}",
        filename=plot_filename_png,
    )

print("\n--- Analysis complete for all pairs. ---")

In [ ]:
methods = df["method_name"].unique()
all_hypervolumes = {}

for method in methods:
    all_hypervolumes[method] = calculate_average_hypervolumes(df, method, perf_metric=perf_metric, hw_metrics=hw_metrics)

# You might want to make this plot function also save to a dynamic filename
plot_hypervolumes(all_hypervolumes, f"bp_hv_{perf_metric}_{hw_metric}")

# Create dynamic filenames to avoid overwriting results
csv_filename = f"../data/hypervolumes_{perf_metric}_hw_metrics.csv"
plot_filename_pdf = f"../plots/CDP_HV_{perf_metric}_hw_metrics.pdf"
plot_filename_png = f"../plots/CDP_HV_{perf_metric}_hw_metrics.png"

hypervolumes_df = pd.DataFrame(all_hypervolumes)
hypervolumes_df.to_csv(csv_filename, index=False)
print(f"Saved hypervolume data to {csv_filename}")

data = []
for method, task_ids in all_hypervolumes.items():
    for task_id, hypervolume in task_ids.items():
        data.append({"Task": task_id, "Method": method, "Hypervolume": hypervolume})

df_hypervolumes = pd.DataFrame(data)
pivot_hypervolumes = df_hypervolumes.pivot(
    index="Task", columns="Method", values="Hypervolume"
)

# Run CD evaluation with dynamic filenames
hv_result = cd_evaluation(
    pivot_hypervolumes,
    maximize_metric=True,
    plt_title=f"Hypervolume CD Plot for {perf_metric} vs {hw_metric}",
    filename=plot_filename_pdf,
)
hv_result = cd_evaluation(
    pivot_hypervolumes,
    maximize_metric=True,
    plt_title=f"Hypervolume CD Plot for {perf_metric} vs {hw_metric}",
    filename=plot_filename_png,
)

### Inverted Generational Distance

In [ ]:
from pymoo.indicators.igd_plus import IGDPlus

def calculate_IGDp_per_method(df, perf_metric: str, hw_metrics: list[str]):
    results = []
    all_objectives = [perf_metric] + hw_metrics

    # Iterate over unique task_ids
    for task_id in df["task_id"].unique():
        # Iterate over unique seeds
        for seed in df["seed"].unique():
            # Iterate over unique folds
            for fold in df["fold"].unique():
                # Filter the DataFrame for the current task_id, seed, and fold
                df_fold = df[
                    (df["task_id"] == task_id)
                    & (df["seed"] == seed)
                    & (df["fold"] == fold)
                ]

                if df_fold.empty:
                    continue  # Skip if no data for this combination

                # Compute the reference Pareto front (from all methods)
                is_efficient = getParetoFront(df_fold, all_objectives, return_mask=True)
                reference_pareto_front = df_fold[is_efficient]
                pf_points = reference_pareto_front[all_objectives].values

                # Iterate over unique method_names
                for method_name in df_fold["method_name"].unique():
                    df_method = df_fold[df_fold["method_name"] == method_name]

                    # Compute Pareto front for df_method
                    is_efficient = getParetoFront(df_method, all_objectives, return_mask=True)
                    method_pareto_front = df_method[is_efficient]
                    method_points = method_pareto_front[all_objectives].values

                    if len(method_points) == 0 or len(pf_points) == 0:
                        igd_value = np.nan
                    else:
                        # Calculate IGD
                        ind = IGDPlus(pf_points)
                        igd_value = ind(method_points)

                    # Append results
                    results.append({
                        "task_id": task_id,
                        "seed": seed,
                        "fold": fold,
                        "method_name": method_name,
                        "IGD": igd_value
                    })

    return pd.DataFrame(results)

In [ ]:
# Calculate IGD+ per method
for perf_metric, hw_metric in metric_pairs:
    metric_pair = [perf_metric, hw_metric]
    igd_results = calculate_IGDp_per_method(df, perf_metric, [hw_metric])
    
    # Group by 'method_name' and 'task_id', and aggregate with 'sum'
    agg_igd = igd_results.groupby(["method_name", "task_id"]).agg('sum').reset_index()
    igd_pivots = agg_igd.pivot(
        index="task_id", columns="method_name", values="IGD"
    )

    igd_result = cd_evaluation(
        igd_pivots,
        maximize_metric=False,
        plt_title="Hypervolume Critical Difference Plot",
        filename=f"../plots/CDP_IGD_{perf_metric}_{hw_metric}.png",
    )
    igd_result = cd_evaluation(
        igd_pivots,
        maximize_metric=False,
        plt_title="Hypervolume Critical Difference Plot",
        filename=f"../plots/CDP_IGD_{perf_metric}_{hw_metric}.pdf",
    )

In [ ]:
igd_results = calculate_IGDp_per_method(df, perf_metric, hw_metrics)
agg_igd = igd_results.groupby(["method_name", "task_id"]).agg('sum').reset_index()
igd_pivots = agg_igd.pivot(
    index="task_id", columns="method_name", values="IGD"
)

igd_result = cd_evaluation(
    igd_pivots,
    maximize_metric=False,
    plt_title="Hypervolume Critical Difference Plot",
    filename=f"../plots/CDP_IGD_{perf_metric}_hw_metrics.png",
)
igd_result = cd_evaluation(
    igd_pivots,
    maximize_metric=False,
    plt_title="Hypervolume Critical Difference Plot",
    filename=f"../plots/CDP_IGD_{perf_metric}_hw_metrics.pdf",
)

### Normalized Improvement

In [ ]:
import matplotlib.ticker as mtick

def plot_normalized_improvement(
    df_scores: pd.DataFrame,
    baseline_method: str,
    higher_is_better: bool,
    title: str,
    directory: str = "../plots/",
):
    # Check if the baseline method exists in the DataFrame
    if baseline_method not in df_scores.columns:
        print(f"Warning: Baseline method '{baseline_method}' not found. Skipping plot.")
        return

    # Extract the baseline scores for comparison
    baseline_scores = df_scores[baseline_method]
    
    # Initialize a new DataFrame to store the improvement values
    improvement_df = pd.DataFrame(index=df_scores.index)

    # Get a list of methods to compare against the baseline
    methods_to_plot = [col for col in df_scores.columns if col != baseline_method]

    for method in methods_to_plot:
        method_scores = df_scores[method]
        
        # Calculate the raw improvement based on whether higher or lower scores are better
        if higher_is_better:
            # Improvement = (Method Score - Baseline Score)
            improvement = method_scores - baseline_scores
        else:
            # Improvement = (Baseline Score - Method Score)
            improvement = baseline_scores - method_scores

        # Normalize the improvement by the absolute value of the baseline score
        # This gives the relative (percentage) improvement.
        # A small epsilon (1e-9) is added to avoid division by zero.
        normalized_improvement = improvement / (np.abs(baseline_scores) + 1e-9)
        improvement_df[method] = normalized_improvement

    # Restructure the DataFrame from wide to long format for easy plotting with seaborn
    melted_df = improvement_df.melt(var_name="Method", value_name="Normalized Improvement")

    # Calculate the median improvement for each method to sort the plot
    median_order = melted_df.groupby("Method")["Normalized Improvement"].median().sort_values(ascending=False).index

    # --- Plotting ---
    plt.figure(figsize=(12, 8))
    sns.set(style="whitegrid")
    ax = sns.boxplot(
        y="Method",
        x="Normalized Improvement",
        data=melted_df,
        hue="Method",
        palette="viridis",
        orient="h",
        order=median_order,
        showfliers=False,  # Outliers can skew the view, so they are hidden for clarity
    )

    # Add a vertical dashed red line at 0% to represent the baseline performance
    ax.axvline(0, color='r', linestyle='--')

    # Set plot titles and labels with appropriate font sizes
    ax.set_title(title, fontsize=22, pad=20)
    ax.set_ylabel("Method", fontsize=18)
    ax.set_xlabel(f"Normalized Improvement Over {baseline_method}", fontsize=18)
    
    # Set tick sizes
    ax.tick_params(axis="x", labelsize=14)
    ax.tick_params(axis="y", labelsize=14)
    
    # Format the x-axis to show percentages
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

    plt.tight_layout()
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # Sanitize the title to create a valid filename
    filename_title = title.replace(' ', '_').replace('/', '_')
    
    # Save the plot in both PNG and PDF formats
    plt.savefig(f"{directory}/{filename_title}.png", dpi=300, bbox_inches="tight")
    plt.savefig(f"{directory}/{filename_title}.pdf", dpi=300, bbox_inches="tight")
    
    plt.show()
    plt.close()

In [ ]:
# --- Plot for Hypervolume ---
# Higher is better for Hypervolume, so we set higher_is_better=True
plot_normalized_improvement(
    df_scores=pivot_hypervolumes,
    baseline_method="Single-Best",
    higher_is_better=True,
    title="Normalized Improvement on Hypervolume vs Single-Best"
)

# --- Plot for IGD+ ---
# Lower is better for IGD+, so we set higher_is_better=False
plot_normalized_improvement(
    df_scores=igd_pivots,
    baseline_method="Single-Best",
    higher_is_better=False,
    title="Normalized Improvement on IGD+ vs Single-Best"
)

### Pareto Front Topography

In [ ]:
# Get unique method names
method_names = df["method_name"].unique()

# Calculate Pareto fronts for each method
method_pareto_fronts = {}
for method in method_names:
    df_method = df[df["method_name"] == method]
    method_pareto_fronts[method] = calculate_pareto_fronts(df_method, ['normalized_roc_auc_test', 'normalized_time'])

# Function to count the number of solutions in Pareto fronts
def count_pareto_solutions(pareto_fronts):
    counts = {}
    for task_id in pareto_fronts:
        for seed in pareto_fronts[task_id]:
            for fold in pareto_fronts[task_id][seed]:
                pareto_front_df = pareto_fronts[task_id][seed][fold]
                n_solutions = len(pareto_front_df)
                key = (task_id, seed, fold)
                counts[key] = n_solutions
    return counts

# Count solutions in the method-specific Pareto fronts
method_pareto_counts = {}
for method in method_names:
    method_pareto_counts[method] = count_pareto_solutions(method_pareto_fronts[method])

# Calculate total number of solutions per method, task_id, seed, fold
total_counts = df.groupby(['method_name', 'task_id', 'seed', 'fold']).size().reset_index(name='total_solutions')

# Prepare data for plotting counts
data_list = []

# Add the method-specific Pareto fronts and total counts
for method in method_names:
    counts = method_pareto_counts[method]
    for key, n_solutions in counts.items():
        task_id, seed, fold = key
        # Get total number of solutions for this method, task_id, seed, fold
        total_solutions = total_counts[
            (total_counts['method_name'] == method) &
            (total_counts['task_id'] == task_id) &
            (total_counts['seed'] == seed) &
            (total_counts['fold'] == fold)
        ]['total_solutions'].values[0]
        data_list.append({
            'method_name': method,
            'task_id': task_id,
            'seed': seed,
            'fold': fold,
            'n_pareto_solutions': n_solutions,
            'total_solutions': total_solutions
        })

pareto_counts_df = pd.DataFrame(data_list)
pareto_counts_df['proportion_pareto'] = pareto_counts_df['n_pareto_solutions'] / pareto_counts_df['total_solutions']


# Calculate the average number of Pareto solutions per method
avg_pareto_solutions = pareto_counts_df.groupby('method_name')['n_pareto_solutions'].mean().reset_index()

# Calculate the average total number of solutions per method
avg_total_solutions = pareto_counts_df.groupby('method_name')['total_solutions'].mean().reset_index()

# Merge the averages into a single DataFrame
avg_counts_df = pd.merge(avg_pareto_solutions, avg_total_solutions, on='method_name')

# Sort methods by average total solutions for better visualization
avg_counts_df = avg_counts_df.sort_values('total_solutions')

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=False)

# Plot average number of Pareto solutions
sns.barplot(
    data=avg_counts_df,
    x='method_name',
    y='n_pareto_solutions',
    ax=axes[0],
    palette='Blues_d'
)
axes[0].set_title(None)
axes[0].set_xlabel(None)
axes[0].set_ylabel('Average Number of Pareto Solutions', fontsize=14)
axes[0].tick_params(axis='x', rotation=90, labelsize=14)
axes[0].tick_params(axis='y', labelsize=14)

# Plot average total number of solutions
sns.barplot(
    data=avg_counts_df,
    x='method_name',
    y='total_solutions',
    ax=axes[1],
    palette='Greens_d'
)
axes[1].set_title(None)
axes[1].set_xlabel(None)
axes[1].set_ylabel('Average Total Number of Solutions', fontsize=14)
axes[1].tick_params(axis='x', rotation=90, labelsize=14)
axes[1].tick_params(axis='y', labelsize=14)

plt.tight_layout()
plt.savefig("../plots/avg_good_pareto_vs_total_ensembles.png", dpi=300)
plt.savefig("../plots/avg_good_pareto_vs_total_ensembles.pdf", dpi=300)
plt.show()

### Proportion of Solutions on the Global Pareto Front

The following plot illustrates the effectiveness of each method in generating solutions that are part of the global Pareto front. For each task, the global Pareto front is determined by considering all solutions from all methods. We then calculate the proportion of each method's generated solutions that lie on this global front. A higher proportion indicates that a method is more efficient at finding globally optimal solutions.

In [ ]:
import matplotlib.ticker as mtick

print("Preparing data for per-task and per-seed analysis...")
df_main = df.copy()
task_column_name = 'task'

cost_metrics = ['normalized_time', 'normalized_memory', 'normalized_diskspace']
unique_tasks = df_main[task_column_name].unique()
unique_seeds = df_main['seed'].unique()
all_proportions = []

# Loop over each task
for task in unique_tasks:
    # --- CHANGE: Added inner loop to iterate over each seed ---
    for seed in unique_seeds:
        # Filter for the specific task and seed combination
        df_task_seed = df_main[(df_main[task_column_name] == task) & (df_main['seed'] == seed)].copy()

        # If there's no data for this combination, skip to the next
        if df_task_seed.empty:
            continue

        # Get total solutions for this specific task-seed combo
        total_solutions_task_seed = df_task_seed.groupby('method_name', observed=True).size()

        for metric in cost_metrics:
            objectives = ['normalized_roc_auc_test', metric]
            pf_col_name = f'is_on_pf_{metric}'
            df_task_seed[pf_col_name] = False

            # Calculate Pareto front for each fold within this task and seed
            global_pareto_indices = []
            # --- CHANGE: Grouping is now only by 'fold' ---
            for name, group in df_task_seed.groupby(['fold']):
                if not group.empty:
                    is_efficient_mask = getParetoFront(group, objectives, return_mask=True)
                    global_pareto_indices.extend(group[is_efficient_mask].index)

            # Ensure we only use unique indices
            df_task_seed.loc[list(set(global_pareto_indices)), pf_col_name] = True

            # Calculate proportions for each method within this task-seed combo
            solutions_on_pf = df_task_seed.groupby('method_name', observed=True)[pf_col_name].sum()
            task_seed_proportions = (solutions_on_pf / total_solutions_task_seed).fillna(0)

            # Store the results for this task-seed combo
            for method_name, proportion in task_seed_proportions.items():
                all_proportions.append({
                    task_column_name: task,
                    'seed': seed, # Also store the seed
                    'method_name': method_name,
                    'metric_type': metric.replace('_', ' ').replace('normalized', '').strip().capitalize(),
                    'proportion': proportion,
                    'num_solutions': total_solutions_task_seed.get(method_name, 0)
                })

print("Averaging the per-task-seed results...")
per_task_seed_df = pd.DataFrame(all_proportions)

# Filter out the 'Single-Best' method if it exists
if 'Single-Best' in per_task_seed_df['method_name'].unique():
    per_task_seed_df = per_task_seed_df[per_task_seed_df['method_name'] != 'Single-Best']

# Forcefully remove the unused 'Single-Best' category to fix plotting issues
if isinstance(per_task_seed_df['method_name'].dtype, pd.CategoricalDtype):
    per_task_seed_df['method_name'] = per_task_seed_df['method_name'].cat.remove_unused_categories()

# Determine the final sort order based on the new, more granular mean
sort_order_df = per_task_seed_df.groupby('method_name', observed=True)['proportion'].mean().sort_values(ascending=False)
sort_order = sort_order_df.index.tolist()
# Note: n is now the average number of solutions per task-seed combination
avg_solutions_per_method = per_task_seed_df.groupby('method_name')['num_solutions'].mean()

print("Generating the final plot...")
plt.figure(figsize=(14, 10))
ax = sns.barplot(
    data=per_task_seed_df,
    y='method_name',
    x='proportion',
    hue='metric_type',
    order=sort_order,
    palette='viridis',
    errorbar='se'
)

new_yticklabels = []
for method in sort_order:
    avg_sol = avg_solutions_per_method.get(method, 0)
    # The label now reflects that 'n' is the avg per task-seed run
    new_yticklabels.append(f"{method}\n(n={avg_sol:.1f})")

ax.set_yticklabels(new_yticklabels)

# Apply formatting
ax.set_xlabel('Average Per-Task Ensemble Proportion on Global Pareto Front', fontsize=18)
ax.set_ylabel('')
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=14)
ax.legend(title='Cost Metric', fontsize=14, title_fontsize=16)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

directory = "../plots"
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(f"{directory}/proportion_on_global_pf_per_task_seed_avg.png", dpi=300, bbox_inches="tight")
plt.savefig(f"{directory}/proportion_on_global_pf_per_task_seed_avg.pdf", dpi=300, bbox_inches="tight")

plt.show()

## Ensemble Construction Behavior

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

objectives_to_plot = ['normalized_roc_auc_test', 'hw_score']

# --- Performance: Subsample the data ---
n_samples = 3000000
if len(df) > n_samples:
    print(f"Dataset is large. Using a random sample of {n_samples} points for speed.")
    df_sample = df.sample(n=n_samples, random_state=42)
else:
    df_sample = df

# --- Method Filtering/Sorting ---
# ['Single-Best', 'Memory', 'QDO-ES', 'Ensemble Size', 'HAPEns', 'Multi-GES', 'GES*', 'Diskspace']
custom_method_order = ['HAPEns', 'Multi-GES', 'Single-Best', 'QDO-ES', 'GES*']
if custom_method_order:
    method_names = [m for m in custom_method_order if m in df_sample['method_name'].unique()]
else:
    method_names = sorted(df_sample['method_name'].unique())

num_methods = len(method_names)

grid_cols = math.ceil(math.sqrt(num_methods))
grid_rows = math.ceil(num_methods / grid_cols)

# --- DYNAMIC FIGURE SIZE CALCULATION ---
subplot_size_inch = 4 
fig_width = grid_cols * subplot_size_inch
fig_height = grid_rows * subplot_size_inch

# --- Plotting ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(
    grid_rows, 
    grid_cols, 
    figsize=(fig_width, fig_height),
    sharex=True, 
    sharey=True
)
axes = axes.flatten()

print("Generating final polished density plots...")

# --- Loop through each method and create its subplot ---
for i, method in enumerate(method_names):
    ax = axes[i]
    df_method_sample = df_sample[df_sample['method_name'] == method]
    
    sns.kdeplot(
        data=df_method_sample,
        x=objectives_to_plot[0],
        y=objectives_to_plot[1],
        fill=True,
        alpha=0.5,
        color='c',
        levels=7,
        ax=ax
    )
    
    sns.kdeplot(
        data=df_method_sample,
        x=objectives_to_plot[0],
        y=objectives_to_plot[1],
        fill=False,
        linewidths=1.0,
        color='teal',
        levels=7,
        ax=ax
    )
    
    ax.set_title(method, fontsize=18, pad=12)
    ax.set_aspect('equal', adjustable='box')

# --- Final Touches ---
# Hide unused subplots
for i in range(num_methods, len(axes)):
    axes[i].set_visible(False)

# Set common labels for the entire figure
fig.supxlabel('Normalized Test ROC AUC (inverted)', fontsize=22, y=0.07)
fig.supylabel('Hardware Score', fontsize=22, x=0.08)

# --- MODIFIED SECTION ---
# Define the desired ticks
major_ticks = [0, 0.25, 0.5, 0.75, 1.0]

# Set shared limits and ticks for all plots
plt.setp(
    axes, 
    xlim=(-0.02, 1.02), ylim=(-0.02, 1.02),
    xticks=major_ticks, yticks=major_ticks
)
# --- END MODIFIED SECTION ---

# Configure labels and ticks for all plots
for i in range(num_methods):
    ax = axes[i]
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='both', labelsize=16)


# Adjust layout
plt.tight_layout(rect=[0.07, 0.07, 1, 1])

plt.savefig("../plots/density_plot.png", dpi=300)
plt.savefig("../plots/density_plot.pdf", dpi=300)
plt.show()

## Latex Table

In [ ]:
def create_latex_table(df, repo, filename="table.tex", max_char=15):
    methods = df["method_name"].unique()
    task_ids = df["task_id"].unique()

    with open(filename, "w") as f:
        f.write("\\begin{longtable}{l" + "c" * len(methods) + "}\n")
        f.write(
            "\\caption{Test ROC AUC - Binary: The mean and standard deviation of the test score over all folds for each method. The best methods per dataset are shown in bold. All methods close to the best method are considered best (using NumPy’s default \\texttt{isclose} function).}\n"
        )
        f.write("\\label{tab:results} \\\\ \n")
        f.write("\\toprule\n")
        f.write("Dataset & " + " & ".join(map(str, methods)) + " \\\\\n")
        f.write("\\midrule\n")
        f.write("\\endfirsthead\n")
        f.write("\\toprule\n")
        f.write("Dataset & " + " & ".join(map(str, methods)) + " \\\\\n")
        f.write("\\midrule\n")
        f.write("\\endhead\n")
        f.write("\\midrule\n")
        f.write(
            "\\multicolumn{"
            + str(len(methods) + 1)
            + "}{r}{Continued on next page} \\\\\n"
        )
        f.write("\\midrule\n")
        f.write("\\endfoot\n")
        f.write("\\bottomrule\n")
        f.write("\\endlastfoot\n")

        for task_id in task_ids:
            dataset_name = repo.tid_to_dataset(
                task_id
            )  # Convert task_id to dataset name
            truncated_name = (
                (dataset_name[:max_char] + "...")
                if len(dataset_name) > max_char
                else dataset_name
            )
            escaped_name = truncated_name.replace("_", "\\_")  # Escape underscores
            line = [str(escaped_name)]  # Ensure the first item is a string
            method_scores = []

            for method in methods:
                method_data = df[
                    (df["task_id"] == task_id) & (df["method_name"] == method)
                ]
                if not method_data.empty:
                    mean_score = method_data["roc_auc_test"].mean()
                    std_dev = method_data["roc_auc_test"].std()
                    score_str = f"{mean_score:.4f}($\\pm${std_dev:.4f})"
                    method_scores.append((mean_score, score_str))
                else:
                    method_scores.append((None, "-"))

            # Determine the best score
            best_score = max(
                score[0] for score in method_scores if score[0] is not None
            )

            for mean_score, score_str in method_scores:
                if mean_score is not None and np.isclose(mean_score, best_score):
                    line.append(f"\\textbf{{{score_str}}}")
                else:
                    line.append(score_str)

            f.write(" & ".join(line) + " \\\\\n")

        f.write("\\bottomrule\n")
        f.write("\\end{longtable}\n")

In [ ]:
directory = "../tables"
if not os.path.exists(directory):
    os.makedirs(directory)

repo = load_repository("D244_F3_C1530_100", cache=True)
create_latex_table(df, repo, filename="../tables/table.tex")